In [2]:
import hyperspy.api as hs
import snmfem.data_analysis as da

In [4]:
s = hs.load("46_ES-LP_L6_brg_test2.spx")

In [5]:
s.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── detector_type = Custom type
│       │       ├── elevation_angle = 22.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 92.449
│       │       └── real_time = 100.677
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       └── beam_energy = 200.0
├── General
│   ├── date = 2020-07-22
│   ├── original_filename = 46_ES-LP_L6_brg_test2.spx
│   ├── time = 16:50:08
│   └── title = EDX
├── Sample
│   ├── elements = ['Fe', 'Si', 'Mg', 'Nd', 'Sm', 'Ca', 'Al', 'Hf', 'Lu', 'U', 'O']
│   └── name = 46_ES-LP_L6_brg_test2
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_TEM